### Prediction part of 1st and 3rd model

In [1]:
# prediction
import numpy as np
import json
from tensorflow.keras.preprocessing.text import Tokenizer
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

text_data = "The feeling you can know so much Without knowing anything at all And now that I can put this down If I had known what I know now \
I never would've played so nonchalant Taxi cabs and busy streets That never bring you back to me I can't help but wish you took me with you And this is when the \
    feeling sinks in I don't wanna miss you like this Come back, be here, come back, be here I guess you're in London today And I don't wanna need you this way \
        Come back, be here, come back, be here You might also like This is falling in love in the cruelest way "
# Initialize tokenizer
tokenizer = Tokenizer()
with open("/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/char-rnn-keras-master/data/char_to_idx_dictcheck_indexreset.json", 'r') as file:
    char_to_idx = json.load(file)
 
vocab_size = len(char_to_idx)
tokenizer.word_index=char_to_idx
T = tokenizer.texts_to_sequences(text_data)
T = np.asarray([lst for lst in T if lst], dtype=np.int32)

from tensorflow.keras.models import load_model
model = load_model("/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/Test/model/model_run4.h5")  # Load the entire model from the file

transposed_input = np.transpose(T[:64], (1, 0))
repeated_input = np.tile(transposed_input, (16, 1))

res = model.predict(repeated_input)
index_word = {index: word for word, index in char_to_idx.items()}
# Apply argmax along the last dimension
argmax_indices = np.argmax(res, axis=-1)
# Convert the argmax indices to words using the index_word dictionary
ind_check = [index for indices in argmax_indices for index in indices]
words = [index_word[index] for indices in argmax_indices for index in indices]
# Print the predicted words
print(words)

2023-06-14 14:15:50.594965: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 14:15:54.266895: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 14:16:03.056412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-14 14:16:26.663426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13508 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:5e:00.0, compute capability: 7.5
20

1/1 [==============================] - 43s 43s/step
['use', 'use', 'discourse', 'i', 'a', 'a', 'doing', 'doing', 'use', 'i', 'a', 'a', 'discourse', 'discourse', 'doing', 'doing', 'doing', 'a', 'a', 'doing', 'doing', 'doing', 'use', 'use', 'doing', 'i', 'i', 'i', 'i', 'use', 'doing', 'i', 'use', 'i', 'i', 'use', 'doing', 'i', 'doing', 'use', 'use', 'doing', 'doing', 'doing', 'doing', 'doing', 'doing', 'discourse', 'use', 'doing', 'discourse', 'use', 'doing', 'constable', 'doing', 'doing', 'use', 'a', 'doing', 'doing', 'a', 'doing', 'i', 'doing', 'use', 'use', 'discourse', 'i', 'a', 'a', 'doing', 'doing', 'use', 'i', 'a', 'a', 'discourse', 'discourse', 'doing', 'doing', 'doing', 'a', 'a', 'doing', 'doing', 'doing', 'use', 'use', 'doing', 'i', 'i', 'i', 'i', 'use', 'doing', 'i', 'use', 'i', 'i', 'use', 'doing', 'i', 'doing', 'use', 'use', 'doing', 'doing', 'doing', 'doing', 'doing', 'doing', 'discourse', 'use', 'doing', 'discourse', 'use', 'doing', 'constable', 'doing', 'doing', 'use', 'a

In [2]:
" ".join(words) 

'use use discourse i a a doing doing use i a a discourse discourse doing doing doing a a doing doing doing use use doing i i i i use doing i use i i use doing i doing use use doing doing doing doing doing doing discourse use doing discourse use doing constable doing doing use a doing doing a doing i doing use use discourse i a a doing doing use i a a discourse discourse doing doing doing a a doing doing doing use use doing i i i i use doing i use i i use doing i doing use use doing doing doing doing doing doing discourse use doing discourse use doing constable doing doing use a doing doing a doing i doing use use discourse i a a doing doing use i a a discourse discourse doing doing doing a a doing doing doing use use doing i i i i use doing i use i i use doing i doing use use doing doing doing doing doing doing discourse use doing discourse use doing constable doing doing use a doing doing a doing i doing use use discourse i a a doing doing use i a a discourse discourse doing doing doi

### Third model

In [1]:
import os
import json
import nltk
import pickle
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm

2023-06-14 15:22:03.574273: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 15:22:03.649042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 15:22:04.925526: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
BATCH_SIZE = 16
SEQ_LENGTH = 64
DATA_DIR = '/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/Test/data'

# Load text data from file
text_data_file_path = '/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/Test/data/input.txt'
with open(text_data_file_path, 'r') as file:
    text_data = file.read().splitlines()
# Initialize tokenizer
tokenizer = Tokenizer()
# Fit tokenizer on text data
tokenizer.fit_on_texts(text_data)
'''
tokenizer_file_path = '/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/Test/model/tokenizer_if_needed.pkl'
with open(tokenizer_file_path, 'wb') as file:
    pickle.dump(tokenizer, file)
'''
# Create a dictionary of unique words
word_index = tokenizer.word_index
# Print the dictionary
#print("All unique words from raw data: ", len(word_index))  #8130
# Download the English word corpus from NLTK
nltk.download('words')
# Get the set of English words
english_words = set(nltk.corpus.words.words())
# Filter out words in word_index that are not valid English words
valid_word_index = {word:index for word,index in word_index.items() if word in english_words}
char_to_idx = {key: index + 1 for index, key in enumerate(valid_word_index)}   
#char_to_idx = valid_word_index 

#print("Number of unique characters: " + str(len(char_to_idx))) #4474

with open(os.path.join(DATA_DIR, 'char_to_idx_new_code.json'), 'w') as f:
    json.dump(char_to_idx, f)

#idx_to_char = { i: ch for (ch, i) in char_to_idx.items() }
vocab_size = len(char_to_idx) #4474

model = Sequential()
model.add(Embedding(vocab_size, 512, batch_input_shape=(BATCH_SIZE, SEQ_LENGTH)))
for i in range(3):
    model.add(LSTM(256, return_sequences=True, stateful=True))
    model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(vocab_size))) 
model.add(Activation('softmax'))
#model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
'''
#Train data generation
text = ' '.join(text_data)
T = tokenizer.texts_to_sequences(text)
#print("Length of text:" + str(T.size)) 
T = np.asarray([lst for lst in T if lst], dtype=np.int32)
print(T.shape) #(506795, 1)
mask = np.all(np.isin(T, list(char_to_idx.values())), axis=1)
T = T[mask]
print(T.shape) #(448858, 1)

length = T.shape[0]; 
X_final = np.array([[]])
Y_final = np.array([[[]]])
batch_chars = int(length / BATCH_SIZE); #28053
for start in tqdm(range(0, batch_chars - SEQ_LENGTH, SEQ_LENGTH)): 
    X = np.zeros((BATCH_SIZE, SEQ_LENGTH)) 
    Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, vocab_size)) 
    for batch_idx in range(0, BATCH_SIZE): # (0,16)
        for i in range(0, SEQ_LENGTH): #(0,64)
            X[batch_idx, i] = T[batch_chars * batch_idx + start + i] # 
            Y[batch_idx, i, T[batch_chars * batch_idx + start + i + 1]] = 1    
    if X_final.size == 0:
        X_final = X
    else:
        X_final = np.concatenate((X_final, X))
    
    if Y_final.size == 0:
        Y_final = Y
    else:
        Y_final = np.concatenate((Y_final,Y))   
'''
# Define callbacks
checkpoint_filepath = "/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/data/models/new_code_next500_test-{loss:.4f}_epoch-{epoch:02d}_{accuracy:.4f}.h5"
checkpoint_callback = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='loss')

#early_stopping_callback = EarlyStopping(patience=5, restore_best_weights=True)
#model.load_weights("/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/Test/actual_data/models/model_chpt_Bidir32_loss_emb512_test_Drop0.5-2.3074_epoch-02_6.6678.h5") 
X_final = np.load('X_final.npy')
Y_final = np.load('Y_final.npy')
# Train the model
model = load_model("/home/narayana.mayya@sapt.local/projects/Antoine_coefficient_modelling/extra_stuff/data/models/new_code_next500-1.0709_epoch-496_0.6326.h5")
model.fit(X_final, Y_final, epochs=2, batch_size=BATCH_SIZE, callbacks=[checkpoint_callback])   
    

[nltk_data] Downloading package words to
[nltk_data]     /home/narayana.mayya@sapt.local/nltk_data...
[nltk_data]   Package words is already up-to-date!
2023-06-14 15:23:06.964008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13644 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5
2023-06-14 15:23:07.638403: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-14 15:23:07.641667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_

Epoch 1/2


2023-06-14 15:24:36.750655: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-14 15:24:36.752626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-14 15:24:36.753985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

438/438 [==============================] - 32s 34ms/step - loss: 1.1092 - accuracy: 0.6182
Epoch 2/2
438/438 [==============================] - 13s 29ms/step - loss: 1.1027 - accuracy: 0.6200


In [42]:
#np.save('X_final.npy', X_final)
#np.save('Y_final.npy', Y_final)

In [3]:
#X_final = np.load('X_final.npy')
#Y_final = np.load('Y_final.npy')
print(X_final.shape)
print(Y_final.shape)

(7008, 64)
(7008, 64, 4474)


In [4]:
#model = load_model("/home/narayana.mayya@sapt.local/projects/new_few_apps_streamlit/streamlit_datafile_links/lyrics/new_code_next500-1.0709_epoch-496_0.6326.h5") 
model.summary() # for 1st and 3rd

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (16, 64, 512)             2290688   
                                                                 
 lstm (LSTM)                 (16, 64, 256)             787456    
                                                                 
 dropout (Dropout)           (16, 64, 256)             0         
                                                                 
 lstm_1 (LSTM)               (16, 64, 256)             525312    
                                                                 
 dropout_1 (Dropout)         (16, 64, 256)             0         
                                                                 
 lstm_2 (LSTM)               (16, 64, 256)             525312    
                                                                 
 dropout_2 (Dropout)         (16, 64, 256)             0

### Model 1 final loss and accuracy
Epoch 1/2
loss = 0.7341960072517395, acc = 0.73046875

Epoch 2/2
loss = 0.7732210755348206, acc = 0.720703125